In [148]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from IPython.display import display as Idisplay
import matplotlib.pyplot as plt
%matplotlib inline

In [158]:
df = pd.read_excel('data.xlsx',sheetname='Protocol-Grafiken',header=0,skiprows=[0,1,2,4,3,6,7])
df.rename(columns={'pH': 'pH - inflow','Temp': 'Temp - inflow','Conductivity': 'Conductivity - inflow','COD':'COD - inflow','BOD':'BOD - inflow','Ntotal':'Ntotal - inflow','NH4-N':'NH4-N - inflow','NO3-N':'NO3-N - inflow','NO2-N':'NO2-N - inflow','PO4-Ptotal ':'PO4-Ptotal - inflow','PO4-P':'PO4-P - inflow','Coloration  (#777)':'Coloration (#777) - 436nm - inflow','Unnamed: 15':'Coloration (#777) - 525nm - inflow','Unnamed: 16':'Coloration (#777) - 620nm - inflow','Flux':'Flux - inflow',' volume compensation with outflow':' volume compensation with outflow','pH.1':'pH - reactor','Conductivity.1':'Conductivity-reactor','pH.2':'pH - outflow','Temp.1':'Temp - outflow','Conductivity.2':'Conductivity - outflow','COD.1':'COD - outflow','BOD.1':'BOD - outflow','Ntotal.1':'Ntotal - outflow','NH4-N.1':'NH4-N - outflow','NO3-N.1':'NO3-N - outflow','NO2-N.1':'NO2-N - outflow','PO4-Ptotal .1':'PO4-Ptotal - outflow','PO4-P.1':'PO4-P - outflow','Coloration (no dilution)     (#777)':'Coloration (#777) - 436nm - outflow','Unnamed: 53':'Coloration (#777) - 525nm - outflow','Unnamed: 54':'Coloration (#777) - 620nm - outflow','Flux.1':'Flux - outflow','Unnamed: 72':'Comment'},inplace=True)
columns = df.columns
new_columns = []

for name in columns:
    new_name = name.replace(' ','_')
    new_columns.append(new_name)
    
df.columns = new_columns


In [159]:
personal = df.loc[:,'Date':'Responsible']
personal['Comment'] = df.loc[:,'Comment']
inflow = df.loc[:,'pH_-_inflow':'_volume_compensation_with_outflow']
reactor = df.loc[:,'Reactor_volume':'Sludge_Loading']
tss_vss = df.loc[:,'Bowl_empty':'loss_on_ignition']
outflow = df.loc[:,'pH_-_outflow':'Flux_-_outflow']
COD_balance = df.loc[:,'Load_Feed':'COD_Removal']
N_balance = df.loc[:,'Load_Feed.1':'COD_Removal.1']

In [141]:
Idisplay(inflow.loc[91:91])
Idisplay(reactor.loc[91:91])
Idisplay(outflow.loc[91:91])

,pH - inflow,Temp - inflow,Conductivity - inflow,COD - inflow,BOD - inflow,Ntotal - inflow,NH4-N - inflow,NO3-N - inflow,NO2-N - inflow,PO4-Ptotal - inflow,PO4-P - inflow,Coloration (#777) - 436nm - inflow,Coloration (#777) - 525nm - inflow,Coloration (#777) - 620nm - inflow,Flux - inflow,volume compensation with outflow
91,4.38,30,770,4428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2740,320


,Reactor volume,pH - reactor,T,Conductivity-reactor,O2,Residence Time,Nutrient dosing (ECSO 5150),Nutrient dosing (trace elements),suspended matter (#630),Excess sludge,Total Suspended Solids,Volatile Suspended Solids,Sludge Loading
91,6.5,8.28,25.1,798,NaN,56.934307,NaN,NaN,NaN,NaN,10.745455,85.402098,0.173708


,pH - outflow,Temp - outflow,Conductivity - outflow,Dosierung Pumpe,COD - outflow,BOD - outflow,Ntotal - outflow,NH4-N - outflow,NO3-N - outflow,NO2-N - outflow,PO4-Ptotal - outflow,PO4-P - outflow,Coloration (#777) - 436nm - outflow,Coloration (#777) - 525nm - outflow,Coloration (#777) - 620nm - outflow,Flux - outflow
91,8.36,26.9,736,8,27.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2648


In [207]:
work = reactor.copy()
work['pH_inflow'] = inflow.loc[:,'pH_-_inflow']
work['pH_outflow'] = outflow.loc[:,'pH_-_outflow']
del work['Nutrient_dosing_(trace_elements)']
for name in work.columns:
    mean = work[name].mean()
    work[name].fillna(mean,inplace=True)

t = []
for name in work.columns:
    try:
        if name not in work.columns:
            continue
        if work[name].var() < 1e-7:
            continue
        
        
        formula = "Q('pH_outflow') ~ Q('pH_inflow')+Q('"+name+"')"
        model = smf.ols(formula,data=work)
        if model.nobs < len(df)/2:
            continue
        
        results = model.fit()
    except (ValueError, TypeError):
        continue
        
    t.append((results.rsquared, name))
t.sort(reverse=True)
for mse,name in t:
    print(name,mse)
        

pH_outflow 1.0
pH_-_reactor 0.262370219661
Conductivity-reactor 0.0715532980062
Total_Suspended_Solids 0.0461250776359
Volatile_Suspended_Solids 0.0344456293167
T 0.0183646145931
Excess_sludge 0.0182118296731
Sludge_Loading 0.010419528
Residence_Time 0.00789772988285
Nutrient_dosing_(ECSO_5150) 0.00534437779513
suspended_matter_(#630) 0.00511736958841
Reactor_volume 0.00353579609349
O2 0.00244802064842
pH_inflow 0.00198351729824


In [211]:
work = reactor.copy()
work['pH_inflow'] = inflow.loc[:,'pH_-_inflow']
work['pH_outflow'] = outflow.loc[:,'pH_-_outflow']
del work['Nutrient_dosing_(trace_elements)']
#for name in work.columns:
#    mean = work[name].mean()
#    work[name].fillna(mean,inplace=True)

formula = "Q('pH_outflow') ~ + Q('pH_-_reactor')+Q('Conductivity-reactor')+Q('Total_Suspended_Solids')+Q('Volatile_Suspended_Solids')"
model = smf.ols(formula,data=work)
res = model.fit()
print(res.summary())


                            OLS Regression Results                            
Dep. Variable:        Q('pH_outflow')   R-squared:                       0.488
Model:                            OLS   Adj. R-squared:                  0.467
Method:                 Least Squares   F-statistic:                     23.61
Date:                Thu, 07 Jan 2016   Prob (F-statistic):           1.28e-16
Time:                        10:18:22   Log-Likelihood:                 21.236
No. Observations:                 130   AIC:                            -30.47
Df Residuals:                     124   BIC:                            -13.27
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------------------
Intercept   

In [165]:
Idisplay(inflow.describe())
Idisplay(reactor.describe())
Idisplay(outflow.describe())

,pH_-_inflow,Temp_-_inflow,Conductivity_-_inflow,COD_-_inflow,BOD_-_inflow,Ntotal_-_inflow,NH4-N_-_inflow,NO2-N_-_inflow,PO4-Ptotal_-_inflow,Coloration_(#777)_-_436nm_-_inflow,Flux_-_inflow,_volume_compensation_with_outflow
count,155.000000,94.000000,151.000000,199.000000,0,105.000000,104.000000,103.000000,104.000000,94.000000,205.000000,138.000000
mean,4.547871,26.837234,903.496689,4921.653266,NaN,75.762857,4.728327,0.067864,20.300192,12.986170,2678.975122,358.108696
std,0.885007,1.729005,148.153923,1666.536118,NaN,33.167808,5.672772,0.107018,11.840713,19.376092,654.449893,389.562054
min,3.400000,20.900000,514.000000,1279.000000,NaN,12.500000,0.015000,0.015000,1.800000,0.288000,0.000000,0.000000
25%,3.875000,26.100000,792.500000,3909.500000,NaN,45.300000,0.251500,0.015000,11.440000,0.606000,2600.000000,100.000000
50%,4.220000,26.800000,933.000000,5212.000000,NaN,85.900000,0.860000,0.015000,20.130000,0.902000,2780.000000,265.000000
75%,5.075000,27.800000,1029.500000,6137.000000,NaN,97.500000,9.805000,0.081500,26.830000,19.900000,3003.333333,500.000000
max,7.010000,31.200000,1256.000000,8018.000000,NaN,160.000000,26.000000,0.740000,72.800000,108.000000,3730.000000,2000.000000


,Reactor_volume,pH_-_reactor,T,Conductivity-reactor,O2,Residence_Time,Nutrient_dosing_(ECSO_5150),Nutrient_dosing_(trace_elements),suspended_matter_(#630),Excess_sludge,Total_Suspended_Solids,Volatile_Suspended_Solids,Sludge_Loading
count,210.000000,139.000000,91.000000,137.000000,2.000000,198.000000,5.000000,0,2.000000,24.000000,208.000000,209.000000,200.000000
mean,6.526190,7.885540,24.481319,956.948905,0.305000,61.262352,0.540000,NaN,1655.000000,1219.583333,9.831501,86.847888,0.195229
std,0.140163,0.331383,1.231793,157.999736,0.233345,32.475670,0.089443,NaN,813.172798,609.572046,2.662520,5.854011,0.088697
min,6.500000,7.080000,21.500000,402.000000,0.140000,41.823056,0.500000,NaN,1080.000000,300.000000,2.944118,65.534466,0.000000
25%,6.500000,7.670000,23.600000,883.000000,0.222500,51.942286,0.500000,NaN,1367.500000,645.000000,8.863739,80.000000,0.130883
50%,6.500000,7.900000,24.500000,955.000000,0.305000,55.980861,0.500000,NaN,1655.000000,1165.000000,10.086957,88.963211,0.202078
75%,6.500000,8.135000,25.400000,1018.000000,0.387500,59.541985,0.500000,NaN,1942.500000,1637.500000,11.093241,92.673993,0.250839
max,8.000000,8.630000,27.300000,1620.000000,0.470000,325.000000,0.700000,NaN,2230.000000,2700.000000,18.460870,94.064443,0.423695


,pH_-_outflow,Temp_-_outflow,Conductivity_-_outflow,Dosierung Pumpe,BOD_-_outflow,Ntotal_-_outflow,NH4-N_-_outflow,NO3-N_-_outflow,PO4-Ptotal_-_outflow,PO4-P_-_outflow,Flux_-_outflow
count,140.000000,92.000000,134.00000,159.000000,0,69.000000,72.000000,76.000000,69.000000,69.00000,203.000000
mean,8.173071,24.603261,944.30597,7.628931,NaN,9.535797,0.445333,12.090947,2.493101,2.40613,2683.284072
std,0.292560,1.498164,160.81172,1.028416,NaN,26.255478,1.994410,30.008642,2.231937,2.21510,515.855991
min,6.730000,21.100000,661.00000,4.000000,NaN,0.800000,0.015000,0.230000,0.050000,0.05000,0.000000
25%,8.060000,23.800000,866.25000,8.000000,NaN,1.000000,0.015000,0.230000,0.962000,0.72000,2566.000000
50%,8.200000,24.700000,939.00000,8.000000,NaN,1.350000,0.019000,0.288000,2.070000,2.17000,2678.000000
75%,8.362500,25.600000,1003.00000,8.000000,NaN,3.370000,0.042750,2.550000,3.360000,3.33000,2903.000000
max,8.720000,28.000000,1610.00000,8.000000,NaN,168.200000,15.300000,144.000000,11.800000,11.70000,4828.000000
